<a href="https://colab.research.google.com/github/Mastermrinal/class_repositries/blob/main/Copy_of_1_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align='center' style='color:purple'>Finding best model and hyper parameter tunning using GridSearchCV and RandomisedSearchCV</h2>

**For iris flower dataset in sklearn library, we are going to find out best model and best hyper parameters using GridSearchCV**

**Step1: Load iris flower dataset**

In [1]:
from sklearn import svm, datasets
import numpy as np
# Loading Iris dataset
iris = datasets.load_iris()

**Step2: Converting data into Dataframe**

In [2]:
import pandas as pd
# adding feature names to coloumn
df = pd.DataFrame(iris.data,columns=iris.feature_names)
# adding target coloumn name
df['flower'] = iris.target
# mapping class to each flower name
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


<h3 style='color:blue'>Approach 1: Use train_test_split and manually tune parameters by trial and error</h3>

**Step3: Train test split**

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# (test_size=0.3) --> 30% testing data and 70% is training data
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

**Step4: Training the SVM model**

In [4]:
# creating an instance of SVM classifier
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
# training the model
model.fit(X_train,y_train)
# testing the model
model.score(X_test, y_test)

0.8888888888888888

<h3 style='color:blue'>Approach 2: Use K Fold Cross validation</h3>

**Step5.1: Manually try supplying models and parameters to cross_val_score function with 5 fold cross validation**

In [5]:
from numpy import mean
# k fold cross validation with cv=5
scores=cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)
# printing scores of each fold
print(scores)
# average score
print(mean(scores))

[1.         1.         0.9        0.96666667 1.        ]
0.9733333333333334


**Step5.2: trying with different parameters**

In [6]:
scores=cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)
print(scores)
print(mean(scores))

[0.96666667 1.         0.96666667 0.96666667 1.        ]
0.9800000000000001


**Step5.3: Again trying with different parameters**

In [8]:
scores=cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target, cv=5)
print(scores)
print(mean(scores))

[0.96666667 1.         0.9        0.96666667 1.        ]
0.9666666666666668


**Step6: Above approach is tiresome and very manual. We can use for loop as an alternative**

In [9]:
# trying different values for kernel value
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
# nested loops for iterating on different values of kernel and C
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

<h3 style='color:blue'>Approach 3: Use GridSearchCV</h3>

**Step7: GridSearchCV does exactly same thing as for loop above but in a single line of code**

In [10]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00173888, 0.00081739, 0.00092444, 0.0008389 , 0.00087576,
        0.000808  ]),
 'std_fit_time': array([6.05209953e-04, 3.71396460e-05, 2.79820948e-05, 4.04704144e-05,
        1.58647098e-05, 2.39180302e-05]),
 'mean_score_time': array([0.00120254, 0.00055423, 0.00058823, 0.00058517, 0.00057158,
        0.00054746]),
 'std_score_time': array([7.41682967e-04, 2.88920037e-05, 1.44233585e-05, 7.28688622e-05,
        1.18631872e-05, 1.00355815e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

**Step7.1: Converting results into dataframe**

In [ ]:
df = pd.DataFrame(clf.cv_results_)
df

**Step7.2: Viewing only the required coloumns**

In [ ]:
df[['param_C','param_kernel','mean_test_score']]

**Step7.3: Viewing best parameters**

In [ ]:
clf.best_params_

**Step7.4: Viewing best score**

In [ ]:
clf.best_score_

**Step8: Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    },
    cv=5,
    return_train_score=False,
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

**Step9: How about different models with different hyperparameters?**

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# Iitialising model_params dictionary.Each model is defined as a key-value pair in the dictionary.
# The key represents the models name, and the value is another dictionary that contains the model
# object and its corresponding hyperparameter options.
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}


**Step9.1: Applying loop on the dictionary**

In [ ]:
scores = []
# model_name is the key and mp is vlaue of the above dictionary
# Example svm is model name and mp is the dictionary value of svm
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

**Based on above, I can conclude that SVM with C=1 and kernel='rbf' is the best model for solving my problem of iris flower classification**